In [1]:
from kan import KAN
from sklearn.model_selection import KFold,cross_val_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
import process as data
import numpy as np

ac_p,label=data.deal()
aac=data.fe()
ctd=data.CTD()
gaac=data.gaac()
X=np.concatenate((aac,gaac,ac_p),axis=1)
print(X.shape)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = KAN(width=[37,3,1], grid=3, k=3)
fold = 1

y=label
y=np.array(y)
X = X.astype(np.float32)
y = y.astype(np.float32)

39
39
(740, 37)
checkpoint directory created: ./model
saving model version 0.0


In [2]:
X.shape,y.shape,X[0],y[0]

((740, 37),
 (740,),
 array([ 0.206     ,  0.034     ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.068     ,  0.206     ,
         0.        ,  0.        ,  0.137     ,  0.068     ,  0.206     ,
         0.        ,  0.        ,  0.068     ,  0.        ,  0.        ,
         0.4827586 ,  0.        ,  0.27586207,  0.        ,  0.2413793 ,
         0.3097381 ,  1.0639765 ,  0.0568791 ,  0.86569893,  0.00607317,
         0.10164909,  1.0124953 , -0.8127152 ,  0.017475  ,  0.11910645,
         0.13605663, -0.3426687 ], dtype=float32),
 0.0)

In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)
X_train = torch.tensor(X_train ,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)

/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":
/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/wangli/anaco

In [4]:
X_train.shape,y_train.shape,X_train[0],y_test[0]

(torch.Size([555, 37]),
 torch.Size([555, 1]),
 tensor([ 0.1000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1000,  0.0000,  0.1000,
          0.2000,  0.4000,  0.0000,  0.0500,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0500,  0.7000,  0.0500,  0.2000,  0.0000,
          0.0500, -0.0806,  0.6479,  0.0078,  0.0510, -0.0031,  0.0118,  0.1250,
          0.6300, -0.0030,  0.0134, -0.2254, -0.3329]),
 tensor([0.]))

In [5]:
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}
X_train.shape

torch.Size([555, 37])

In [6]:
model = KAN(width=[37,5,1], grid=7, k=9, seed=42)

checkpoint directory created: ./model
saving model version 0.0


In [7]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    pred=model(X_test)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_test.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_test.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_test.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC(test): {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=30, metrics=(train_acc, test_acc));
# print(results['train_acc'][-1], results['test_acc'][-1])

description:   0%|                                                           | 0/30 [00:00<?, ?it/s]

ACC: 0.9189, AUC: 0.9768, SP: 0.9060, MCC: 0.8376, SN: 0.9308
ACC: 0.9189, AUC(test): 0.9650, SP: 0.9135, MCC: 0.8364, SN: 0.9259


| train_loss: 2.37e-01 | test_loss: 2.66e-01 | reg: 0.00e+00 | :   3%| | 1/30 [13:05<6:19:29, 785.17

ACC: 0.9495, AUC: 0.9859, SP: 0.9286, MCC: 0.8994, SN: 0.9689
ACC: 0.9135, AUC(test): 0.9581, SP: 0.9038, MCC: 0.8261, SN: 0.9259


| train_loss: 1.97e-01 | test_loss: 2.68e-01 | reg: 0.00e+00 | :   7%| | 2/30 [24:49<5:44:11, 737.55

ACC: 0.9766, AUC: 0.9919, SP: 0.9699, MCC: 0.9531, SN: 0.9827
ACC: 0.8919, AUC(test): 0.9160, SP: 0.8846, MCC: 0.7823, SN: 0.9012


| train_loss: 1.62e-01 | test_loss: 3.23e-01 | reg: 0.00e+00 | :  10%| | 3/30 [36:27<5:23:53, 719.76

ACC: 0.9910, AUC: 0.9947, SP: 0.9850, MCC: 0.9820, SN: 0.9965
ACC: 0.8919, AUC(test): 0.9021, SP: 0.9038, MCC: 0.7804, SN: 0.8765


| train_loss: 1.35e-01 | test_loss: 3.34e-01 | reg: 0.00e+00 | :  13%|▏| 4/30 [55:58<6:03:52, 839.73


KeyboardInterrupt: 

In [8]:
def calculate_GAAC(protein_list):
    # 定义氨基酸分组
    groups = {
        'G1': 'GAVLMI',  # 脂肪族
        'G2': 'FYW',     # 芳香族
        'G3': 'KRH',     # 带正电
        'G4': 'DE',      # 带负电
        'G5': 'STCPNQ'   # 不带电
    }

    # 初始化结果列表
    results = []

    # 遍历蛋白质序列
    for protein in protein_list:
        # 计算每个组的频率
        group_counts = {key: 0 for key in groups}
        total_length = len(protein)
        
        for aa in protein:
            for group_name, group_aa in groups.items():
                if aa in group_aa:
                    group_counts[group_name] += 1
                    break
        
        # 计算频率
        group_frequencies = {key: count / total_length for key, count in group_counts.items()}
        results.append(group_frequencies)

    return results

# 示例蛋白质序列列表
protein_list = ["GAVLMIKRHDE", "STCPNQFYWG", "GAVLMIKRHDEGAVLMIKRHDE"]
gaac_features = calculate_GAAC(protein_list)

for i, features in enumerate(gaac_features):
    print(f"Protein {i + 1}: {features}")


Training Accuracy: 0.90
Training AUC: 0.96
Test Accuracy: 0.83
Test AUC: 0.91


/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
